In [2]:
from process import event_preprocess, argument_preprocess,load_event, load_argument, Schema2Label
from src.seqlabel_utils import pos2bio
from src.preprocess.str_utils import get_useless_chars
from process import text_preprocess, gen_pos
from src.dataset.converter import single_text, double_text
import pandas as pd 
import ast 
import numpy as np 

In [3]:
train_e = load_event('./trainsample/duee_train.json')
valid_e = load_event('./trainsample/duee_dev.json')
test_e = load_event('./trainsample/duee_test2.json')
schema = Schema2Label('./trainsample/duee_event_schema.json')
train_e.shape, valid_e.shape, test_e.shape

((11908, 3), (1492, 3), (34904, 2))

In [3]:
useless_chars = get_useless_chars(train_e['text'].values.tolist() +\
                                  valid_e['text'].values.tolist() +\
                                  test_e['text'].values.tolist())

## pipeline: EventType + Arguments

In [4]:
train_e = event_preprocess(train_e, useless_chars)
valid_e = event_preprocess(valid_e, useless_chars)
test_e = event_preprocess(test_e, useless_chars)

0 out of 11908 trigger not match
0 out of 11908 even pos exceed text line
0 out of 1492 trigger not match
0 out of 1492 even pos exceed text line


### 方案1：Trigger抽取序列标注问题

In [ ]:
from src.dataset.converter import single_text
single_text(train_e['id'].values, train_e['clean_text'].values, train_e['event_pos'].values,
            './trainsample','train_event_bio')
single_text(valid_e['id'].values, valid_e['clean_text'].values, valid_e['event_pos'].values, 
            './trainsample','valid_event_bio')
single_text(test_e['id'].values, test_e['clean_text'].values, None ,'./trainsample','test_event_bio')

### 事件抽取方案2: 多标签分类问题

In [ ]:
single_text(train_e['id'].values, train_e['clean_text'].values, train_e['event_label'].values,
            './trainsample','train_event_cls')
single_text(valid_e['id'].values, valid_e['clean_text'].values, valid_e['event_label'].values, 
            './trainsample','valid_event_cls')
single_text(test_e['id'].values, test_e['clean_text'].values, None, './trainsample','test_event_cls')

### 事件抽取方案3：column selection

In [9]:
len(schema.event_label)

65

In [15]:
schema.event_label

{'财经/交易-出售/收购': 0,
 '财经/交易-跌停': 1,
 '财经/交易-加息': 2,
 '财经/交易-降价': 3,
 '财经/交易-降息': 4,
 '财经/交易-融资': 5,
 '财经/交易-上市': 6,
 '财经/交易-涨价': 7,
 '财经/交易-涨停': 8,
 '产品行为-发布': 9,
 '产品行为-获奖': 10,
 '产品行为-上映': 11,
 '产品行为-下架': 12,
 '产品行为-召回': 13,
 '交往-道歉': 14,
 '交往-点赞': 15,
 '交往-感谢': 16,
 '交往-会见': 17,
 '交往-探班': 18,
 '竞赛行为-夺冠': 19,
 '竞赛行为-晋级': 20,
 '竞赛行为-禁赛': 21,
 '竞赛行为-胜负': 22,
 '竞赛行为-退赛': 23,
 '竞赛行为-退役': 24,
 '人生-产子/女': 25,
 '人生-出轨': 26,
 '人生-订婚': 27,
 '人生-分手': 28,
 '人生-怀孕': 29,
 '人生-婚礼': 30,
 '人生-结婚': 31,
 '人生-离婚': 32,
 '人生-庆生': 33,
 '人生-求婚': 34,
 '人生-失联': 35,
 '人生-死亡': 36,
 '司法行为-罚款': 37,
 '司法行为-拘捕': 38,
 '司法行为-举报': 39,
 '司法行为-开庭': 40,
 '司法行为-立案': 41,
 '司法行为-起诉': 42,
 '司法行为-入狱': 43,
 '司法行为-约谈': 44,
 '灾害/意外-爆炸': 45,
 '灾害/意外-车祸': 46,
 '灾害/意外-地震': 47,
 '灾害/意外-洪灾': 48,
 '灾害/意外-起火': 49,
 '灾害/意外-坍/垮塌': 50,
 '灾害/意外-袭击': 51,
 '灾害/意外-坠机': 52,
 '组织关系-裁员': 53,
 '组织关系-辞/离职': 54,
 '组织关系-加盟': 55,
 '组织关系-解雇': 56,
 '组织关系-解散': 57,
 '组织关系-解约': 58,
 '组织关系-停职': 59,
 '组织关系-退出': 60,
 '组织行为-罢工': 61,
 '组织行为-闭幕': 62,
 '组织行为-开幕'

In [19]:
valid_e['text'].map(lambda x: len(x)).describe()

count    1492.000000
mean       58.218499
std        45.147101
min         6.000000
25%        27.000000
50%        38.000000
75%        79.000000
max       333.000000
Name: text, dtype: float64

In [18]:
train_e['text'].map(lambda x: len(x)).describe()

count    11908.000000
mean        57.011253
std         42.758747
min          6.000000
25%         27.000000
50%         38.000000
75%         77.000000
max        378.000000
Name: text, dtype: float64

In [17]:
test_e['text'].map(lambda x: len(x)).describe()

count    34904.000000
mean        49.621619
std         36.864349
min          5.000000
25%         26.000000
50%         34.000000
75%         64.000000
max        564.000000
Name: text, dtype: float64

In [ ]:
enhancement = ''
pos = 1 
pos_list = [pos]

enhancement = []
for event in schema.event_label:
    event = event.split('-')[1]
    enhancement.append('[unused1]' + event + '[unused2]')
    pos+=len(event)+2
    pos_list.append(pos)
enhancement = ''.join(enhancement)   
print(','.join([str(i) for i in pos_list[:-1]]))
print(enhancement)

In [ ]:
510-269

In [ ]:
valid_e['text'].map(lambda x: len(x)>241).sum()

In [ ]:
double_text(train_e['id'].values, [enhancement] * train_e.shape[0], 
            train_e['clean_text'].values, train_e['event_label'].values,
            './trainsample','train_event_slot')
double_text(valid_e['id'].values, [enhancement] * valid_e.shape[0],
            valid_e['clean_text'].values, valid_e['event_label'].values, 
            './trainsample','valid_event_slot')
double_text(test_e['id'].values, [enhancement] * test_e.shape[0], 
            test_e['clean_text'].values, None, './trainsample','test_event_slot')

### Argument抽取

In [13]:
train_a = load_argument('./trainsample/duee_train.json')
valid_a = load_argument('./trainsample/duee_dev.json')
train_a.shape, valid_a.shape, test_a.shape

1,8,12,16,20,24,28,32,36,40,44,48,52,56,60,64,68,72,76,80,84,88,92,96,100,104,110,114,118,122,126,130,134,138,142,146,150,154,158,162,166,170,174,178,182,186,190,194,198,202,206,212,216,220,224,230,234,238,242,246,250,254,258,262,266
[unused1]出售/收购[unused2][unused1]跌停[unused2][unused1]加息[unused2][unused1]降价[unused2][unused1]降息[unused2][unused1]融资[unused2][unused1]上市[unused2][unused1]涨价[unused2][unused1]涨停[unused2][unused1]发布[unused2][unused1]获奖[unused2][unused1]上映[unused2][unused1]下架[unused2][unused1]召回[unused2][unused1]道歉[unused2][unused1]点赞[unused2][unused1]感谢[unused2][unused1]会见[unused2][unused1]探班[unused2][unused1]夺冠[unused2][unused1]晋级[unused2][unused1]禁赛[unused2][unused1]胜负[unused2][unused1]退赛[unused2][unused1]退役[unused2][unused1]产子/女[unused2][unused1]出轨[unused2][unused1]订婚[unused2][unused1]分手[unused2][unused1]怀孕[unused2][unused1]婚礼[unused2][unused1]结婚[unused2][unused1]离婚[unused2][unused1]庆生[unused2][unused1]求婚[unused2][unused1]失联[unused2][unused1]死亡[unused2][unused1]罚款[unused2][

In [20]:
510-269

241

In [22]:
valid_e['text'].map(lambda x: len(x)>241).sum()

8

In [7]:
double_text(train_e['id'].values, [enhancement] * train_e.shape[0], 
            train_e['clean_text'].values, train_e['event_label'].values,
            './trainsample','train_event_slot')
double_text(valid_e['id'].values, [enhancement] * valid_e.shape[0],
            valid_e['clean_text'].values, valid_e['event_label'].values, 
            './trainsample','valid_event_slot')
double_text(test_e['id'].values, [enhancement] * test_e.shape[0], 
            test_e['clean_text'].values, None, './trainsample','test_event_slot')

### Argument抽取

In [ ]:
train_a = load_argument('./trainsample/duee_train.json')
valid_a = load_argument('./trainsample/duee_dev.json')
train_a.shape, valid_a.shape, test_a.shape

In [ ]:
train_a = argument_preprocess(train_a, useless_chars)
valid_a = argument_preprocess(valid_a, useless_chars)

In [ ]:
test_a = pd.read_csv('./trainsample/test_event_pred.csv',index_col=None)
test_a['pred'] = test_a['pred'].map(lambda x: ast.literal_eval(x))
test_a['pred_label'] = test_a['pred'].map(lambda x: extract_multilabel(x, {j:i for i,j in schema.event_label.items()}, 0.5, greedy=True))
test_a = test_a.explode('pred_label')
test_a['event_text'] = test_a.apply(lambda x: x.pred_label + '-' + x.text_clean, axis=1)


In [ ]:
single_text(train_a['id'].values, train_a['event_text'].values, train_a['argument_bio_label'].values,
            './trainsample','train_argument')
single_text(valid_a['id'].values, valid_a['event_text'].values, valid_a['argument_bio_label'].values, 
            './trainsample','valid_argument')
single_text(test_a['id'].values, test_a['event_text'].values, None, './trainsample','test_argument')